%md  
# Summary
The top SAS functions are:    
- Character    
- Currency    
- Financial (not visited)  
- Trigonometric (not visited)  
- Descriptive statistics (not visited)   
- Date/time/temporal     
  
Data manipulation  
1. Convert datetime to date  
2. Extract substrings of a string/char  
3. Switch string orders  
4. Flag mispelled strings  
5. Date difference operations  
6. Calculate total compensation and prettify output (via commas, currency symbols, and rounding)  


## Convert datetime to date

In a pretend table we are first converting the date column to a sas date using date9. which means 9 bytes. THen we are converting the totalcomp column to dollar15.

SAS code

    proc print;    
        format newteam /* No format applied to this variable */     
            newname /* No format applied to this variable */     
            date9.  /* The date9. format is applied to the date variable */    
            totalcomp dollar15.; /* The dollar15. format is applied to the totalcomp variable */    
    run;

Python conversion

    import pandas as pd  
    
    # Load the dataset into a pandas DataFrame (replace 'my_dataset.csv' with your file name)  
    df = pd.read_csv('my_dataset.csv')  
    
    # Apply formatting to the columns in the DataFrame  
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%d%b%Y') # Apply date format  
    df['totalcomp'] = df['totalcomp'].apply(lambda x: '${:,.2f}'.format(x)) # Apply dollar format  
    
    # Print the formatted DataFrame  
    print(df[['newteam', 'newname', 'date', 'totalcomp']])  


## Extract substrings of a string/char  

In a pretend table there is an ID column called "idno". The last character of the string value indicates the division name. 

SAS code

    idno_converted = put (idno, 4.); /* 4. is 4 bytes */
    division_extract  = substr(idno_converted, 4, 1); /* 4th position and 1 byte or 1 character length of the substring to extract */

    if division_extract = '1' then divname = "finance";
        else if division_extract = '2' then divname = 'legal'
            else if division_extract = '3' then divname = 'hr'

Python conversion 
  
    # Convert idno to a string with a length of 4 characters  
    df['idno_converted'] = df['idno'].apply(lambda x: str(x).zfill(4))  
    
    # Extract the last character of the idno_converted column  
    df['division_extract'] = df['idno_converted'].str[-1]  
    
    # Assign division names based on the value of division_extract  
    df.loc[df['division_extract'] == '1', 'divname'] = 'finance'  
    df.loc[df['division_extract'] == '2', 'divname'] = 'legal'  
    df.loc[df['division_extract'] == '3', 'divname'] = 'hr'  

## Switch string orders

In a pretend table there is a name column with the values that are last name, first name (ex: Kraft, Claire). We need to create two columns to split out the last and first names. Then we need to concat the split names into the first name, last name (ex: Claire Kraft)

SAS code

    /* name = 'Kraft, Claire' */ 
    lname = scan(name, 1, ' , '); /* extract last name - Kraft */    
    fname = scan(name, 2, ' , '); /* extract first name - Claire */    
    proper_name = catx(' ', fname, lname) /* concatonate the first name and last name columns with no commas - Claire Kraft */  


Python conversion

    import pandas as pd  

    # Extract the first and last names using the 'str.split' method  
    df['lname'] = df['name'].str.split(', ').str[0] # extract last name  
    df['fname'] = df['name'].str.split(', ').str[1] # extract first name  
    
    # Concatenate the first and last names together with a space separator  
    df['proper_name'] = df['fname'] + ' ' + df['lname']  
    
    # Print the resulting DataFrame  
    print(df[['name', 'proper_name']])  



## Flag mispelled strings

In a pretend table there is a name value that is wrong. The sas code will flag the mispelling.

SAS code

    /*
    FIND function is used to search for a specific string or pattern within a character variable. In this case, the proper_name variable is being searched.
    The first argument of FIND function is the variable to search, which is proper_name.
    The second argument of FIND function is the string or pattern to search for, which is 'Lau'.
    The third argument of FIND function is optional and specifies the starting position of the search. In this case, i is used to indicate a case-insensitive search, which means that both upper- and lowercase versions of 'Lau' will be found.

    0 checks if the FIND function returns a value greater than zero, which indicates that the search string was found within proper_name.
    If the search string is found within proper_name, the PUT statement outputs a message to the SAS log that says 'value found' followed by the value of proper_name.
    The PUT statement is used to write a message to the SAS log or to an output file.
    In this case, the message is a string literal ('value found') followed by the value of the proper_name variable.
    */

    if find(proper_name, 'Lau', 'i) > 0 then
        put 'value found ' proper_name;


Python conversion

    # Import the fuzzywuzzy library for string matching  
    from fuzzywuzzy import fuzz  
    
    # Define the target string and its misspelled variants  
    target_string = 'Lau'  
    misspellings = ['Lah', 'La', 'Lao', 'Lay', 'Luu']  
    
    # Define a function to check for a match between a string and a list of strings  
    def check_match(string, string_list):  
        for s in string_list:  
            if fuzz.ratio(string, s) > 80:  # Set a threshold for a match  
                return True  
        return False  
    
    # Search for the target string and its misspelled variants within the proper_name column  
    for name in df['proper_name']:  
        if target_string in name or check_match(name.lower(), misspellings):  
            print(f'Misspelling found: {name}')  





## Date difference operations

To determine the difference in years between a hire date and term date, you can follow these steps:

1. Extract the date portion of the hire date using the _DATEPART function_.
2. Convert the term date to a numeric date using the _INPUT function_. For example, if the term date is '23may2003', you can convert it to a numeric date which is 15848. This ,15848, is the number of days since January 1, 1960.
3. Calculate the difference in days between the hire date and term date using the _DATDIF function_

SAS code

    newhire = datepart(hiredate); /* Not formated as sas date */
    newterm = input (termdate, date9.);

    /* ACT/ACT is a method of calculating the time between two dates in finance. It is also known as the Actual/Actual day count convention. This method is used to calculate the difference in days between two dates based on the actual number of days in each month and year. */

    if newterm ne . then do; /* ne means not equal or not null */
        tenure_days = datdif(newhire, newterm, 'ACT/ACT'); 
        tenure_years = yrdif(newhire, newterm, 'ACT/ACT');
    end;

Python conversion

    import pandas as pd  
    from datetime import datetime  
    from dateutil.relativedelta import relativedelta  
    
    # Convert hiredate to datetime object  
    df['hiredate'] = pd.to_datetime(df['hiredate'])  
    
    # Extract date part of hiredate  
    df['newhire'] = df['hiredate'].dt.date  
    
    # Convert termdate to datetime object  
    df['newterm'] = pd.to_datetime(df['termdate'], format='%d%b%Y')  
    
    # Calculate tenure_days and tenure_years using ACT/ACT method  
    df['tenure_days'] = (df['newterm'] - df['newhire']).dt.days  
    df['tenure_years'] = (df['newterm'] - df['newhire']) / relativedelta(years=1)  


## Calculate total compensation and prettify output (via commas, currency symbols, and rounding)

Calculate total compensation by adding salary + bonus + merit
- _SUM function_
- _ROUND function_

SAS code

    totalcomp = sum(salary, bonus, merit)
    totalcomp_rounded = round(totalcomp) /* rounds to the nearest dollar value */

Python conversion

    import pandas as pd  

    # Calculate total compensation  
    df['totalcomp'] = df['salary'] + df['bonus'] + df['merit']  
    
    # Round total compensation to nearest dollar value  
    df['totalcomp_rounded'] = df['totalcomp'].round()  
    
    # Print first 5 rows of DataFrame  
    print(df.head())  



## All together

SAS code

    data data_manipulation_func;
        set info;
        ...
    run;

Python conversion

    def date_manipulation_func:
        ...
    